# 合成数据质量评估
具体实现 关键要素
1. Prompt编写质量
2. 批量评分请求函数 jsonl request，获取 数据质量评分
3. 加载，构造 请求文件 jsonl

# 批量推理任务，最佳实践
1. 准备批量推理文件 jsonl
2. TOS上传数据

In [5]:
quality_evaluation_prompt = {
    "v1": {
        "role": "system",
        "content": """
作为半导体显示领域的专业质量评估专家，请严格按以下标准评估问答对的质量。评估分为核心维度，每个维度包含具体评估点和示例参考。

### 评估维度
1. 思维链逻辑质量（权重35%）
   - 步骤完整性：推理步骤是否覆盖问题所有关键点？是否遗漏必要环节？
   - 因果连贯性：前后步骤是否存在清晰因果关系？有无逻辑断裂？
   - 技术参数合理性：工艺参数是否符合物理规律？（例：LTPS退火温度不能超过玻璃转化点）
   - 错误回溯机制：是否考虑可能故障点？（例：分析Mura缺陷应包含设备精度因素）

2. 技术准确度（权重30%）
   - 材料特性：材料描述是否符合物性？（例：IGZO迁移率范围是否正确）
   - 制程参数：工艺参数是否匹配行业标准？（例：光刻精度是否满足当前产线能力）
   - 标准引用：是否准确引用SEMI/SID等国际标准？
   - 专利技术：技术方案是否规避近期专利？（自动核对2020-2024专利数据库）

3. 领域深度（权重20%）
   - 缺陷机理：是否分析根本原因？（例：亮暗点应关联电致迁移机制）
   - 技术趋势：是否覆盖最新发展？（例：需提及Micro LED巨量转移技术）
   - 工艺瓶颈：是否识别关键限制？（例：指出QD-OLED的喷墨打印精度瓶颈）

4. 应用价值（权重15%）
   - 工程可行性：方案是否具备量产实施条件？
   - 成本优化：是否量化成本效益？（例：应计算采用MMG技术的材料节省）
   - 良率提升路径：是否提供可验证的改善方案？

### 领域关键点（自动核对）
| 要素类型       | 典型内容示例                  |
|----------------|------------------------------|
| 核心材料       | 氧化物TFT, QD材料, LTPO      |
| 工艺痛点       | 蒸镀均匀性, 水氧阻隔率       |
| 典型缺陷       | Mura, 亮点/暗点, 热应力翘曲   |

### 验证方法
1. 参数边界检查：对关键参数进行物理极限校验（例：若声称PPI>1500需验证光学混色距离）
2. 时效性验证：技术指标是否被近3年文献更新（自动索引IEEE期刊数据库）
3. 成本分解：对降本承诺进行材料/设备/良率因子分解

### 输出格式要求（JSON）
{
    "quality_rating": {
        "overall": "high/medium/low",
        "detailed_scores": {
            "reasoning_chain": {"score": int, "issues": [str]},
            "technical_accuracy": {"score": int, "validated": bool},
            "domain_depth": {"score": int, "benchmark": str}
        }
    },
    "improvement_suggestions": [str]
}

### 待评估样本
问题: {question_text}
思维链: {reasoning_chain}
答案: {answer_text}
"""
    },
    "v2": {
        "role": "system",
        "content": """你是一名资深显示技术领域专家。请严格评估以下显示技术相关的问答对是否适合用于监督微调（SFT）的数据集构建。评估需基于以下四个核心维度：

1.  **回答相关性 (Relevance)**：回答是否精准聚焦问题核心？是否存在答非所问、偏离主题或遗漏关键点？
2.  **逻辑一致性 (Logical Consistency)**：回答的推理过程是否清晰、连贯、无矛盾？是否存在逻辑跳跃、断裂或自相矛盾？
3.  **术语使用 (Terminology Usage)**：专业术语的使用是否准确、恰当、完整？是否存在术语误用、滥用、缺失或概念性错误？
4.  **事实正确性 (Factual Correctness)**：回答中的技术细节、参数、原理、行业现状等是否符合已知事实和行业共识？是否存在事实性错误或过时信息？

**总体质量评分标准：**
*   `low`：**存在严重缺陷**（如明显事实错误、完全偏离主题、逻辑混乱、关键术语错误），**不适合**用于SFT。
*   `medium`：**存在轻微问题或可优化项**（如部分表述不清、个别术语不严谨、次要逻辑不完美、相关性略有不足），需修改后方可考虑使用。
*   `high`：**无明显错误**，内容**准确、专业、逻辑清晰、紧扣主题**，**适合**直接用于SFT。

**你的任务：**
1.  对每个维度进行独立评分 (`high`/`medium`/`low`)。
2.  给出基于四个维度的**总体质量评分** (`high`/`medium`/`low`)。
3.  对于评分非`high`的维度，**必须具体指出**存在的问题及其**类型**（例如：“术语误用：将‘OLED’错误称为‘LED’”；“事实错误：声称当前主流Mini-LED背光分区数普遍超过5000区”）。
4.  基于你的专业知识和评估结果，**提供具体、可操作的改进建议**，以提升该问答对的质量。

**输出格式要求（严格遵循JSON）：**
{
    "quality_rating": {
        "overall": "high/medium/low", // 总体质量评分
        "detailed_scores": {
            "Relevance": {"score": "high/medium/low", "issues": ["具体问题描述1", "具体问题描述2", ...]}, // 如无问题，issues为空数组[]
            "Logical Consistency": {"score": "high/medium/low", "issues": [...]},
            "Terminology Usage": {"score": "high/medium/low", "issues": [...]},
            "Factual Correctness": {"score": "high/medium/low", "issues": [...]}
        }
    },
    "improvement_suggestions": ["具体建议1", "具体建议2", ...] // 即使总体是high，也可提供优化建议
}
### 待评估样本
问题: {question_text}
思维链: {reasoning_chain}
答案: {answer_text}
"""
    }
}

In [ ]:
import pandas as pd
import json

def excel_to_jsonl(input_excel_path, output_jsonl_path, prompt_version="v1"):
    # 读取Excel文件
    df = pd.read_excel(input_excel_path)
    
    # 确保所需的列存在
    required_columns = ['问题', '推理', '回答']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"缺少必要列: {col}")
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for idx, row in df.iterrows():
            question = row["问题"]
            chain = row["推理"]
            answer = row["回答"]
            quality_message = quality_evaluation_prompt[prompt_version].copy()
            quality_message["content"] = quality_message["content"].replace("{question_text}", question) \
                .replace("{reasoning_chain}", chain) \
                .replace("{answer_text}", answer)

            # 构建请求体
            request_body = {
                "custom_id": f"request-{idx + 1}",
                "body": {
                    "messages": [
                        quality_message
                    ]
                }
            }
            
            # 写入jsonl文件
            f.write(json.dumps(request_body, ensure_ascii=False) + '\n')
def get_request_jsonl(input_excel, output_jsonl):
    excel_to_jsonl(input_excel, output_jsonl, prompt_version="v2")
    print(f"转换完成，结果已保存到 {output_jsonl}")
# # 使用示例
# if __name__ == "__main__":
#     input_excel = "input.xlsx"  # 替换为你的Excel文件路径
#     output_jsonl = "output.jsonl"  # 输出文件路径
#     excel_to_jsonl(input_excel, output_jsonl)
#     print(f"转换完成，结果已保存到 {output_jsonl}")

In [7]:
input_excel = "outputs_excel/syndata_v2.xlsx"  # 替换为你的Excel文件路径
output_jsonl = "outputs_excel/quality_eval_request02.jsonl"  # 输出文件路径
get_request_jsonl(input_excel, output_jsonl)

转换完成，结果已保存到 outputs_excel/quality_eval_request02.jsonl


In [4]:
# input_file_name-文件的完整路径
def valid_request_file(request_filepath):
    with open(request_filepath, 'r', encoding='utf-8') as file:
        total = 0
        custom_id_set = set()
        for line in file:
            if line.strip() == '':
                continue
            try:
                line_dict = json.loads(line)
            except json.decoder.JSONDecodeError:
                raise Exception(f"批量推理任务失败，第{total+1}行非json数据")
            if not line_dict.get('custom_id'):
                raise Exception(f"批量推理任务失败，第{total+1}行custom_id不存在")
            if line_dict.get('custom_id') in custom_id_set:
                raise Exception(f"批量推理任务失败，custom_id={line_dict.get('custom_id', '')}存在重复")
            else:
                custom_id_set.add(line_dict.get('custom_id'))
            if not isinstance(line_dict.get('body', ''), dict):
                raise Exception(f"批量推理任务失败，custom_id={line_dict.get('custom_id', '')}的body非json字符串")
            total += 1
    print(f"total: {total}")
    return total
request_filepath = "outputs_excel/quality_eval_request01.jsonl"
request_filepath = "outputs_excel/quality_eval_request02.jsonl"
valid_request_file(request_filepath)

total: 42


42

In [13]:
import json
import re

def extract_json_from_text(text):
    """
    从可能包含前缀的文本中提取有效的JSON部分
    """
    # 尝试直接解析整个文本
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    
    # 如果直接解析失败，尝试找到第一个{和最后一个}之间的内容
    try:
        # 查找第一个{和最后一个}
        first_brace = text.find('{')
        last_brace = text.rfind('}')
        
        if first_brace != -1 and last_brace != -1 and first_brace < last_brace:
            json_str = text[first_brace:last_brace+1]
            return json.loads(json_str)
    except json.JSONDecodeError:
        pass
    
    # 如果上述方法失败，尝试更宽松的提取方式
    try:
        # 使用正则表达式匹配类似JSON的结构
        json_match = re.search(r'\{[\s\S]*\}', text)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
    except json.JSONDecodeError:
        pass
    
    # 如果所有方法都失败，返回None
    return None

def parse_response_jsonl(input_jsonl_path, output_jsonl_path):
    with open(input_jsonl_path, 'r', encoding='utf-8') as infile, \
         open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
        
        for line in infile:
            try:
                # 解析JSONL中的每一行
                data = json.loads(line.strip())
                
                # 检查是否有response.body.choices[0].message.content
                if data.get('response', {}).get('body', {}).get('choices', [{}])[0].get('message', {}).get('content'):
                    content_str = data['response']['body']['choices'][0]['message']['content']
                    
                    try:
                        # 尝试解析content字段中的JSON
                        # content_json = json.loads(content_str)
                        # 尝试提取和解析JSON
                        content_json = extract_json_from_text(content_str)
                        
                        if content_json is not None:
                            # 写入新的JSONL文件
                            outfile.write(json.dumps(content_json, ensure_ascii=False) + '\n')
                        else:
                            print(f"警告: 无法解析content字段为JSON (custom_id: {data.get('custom_id')})")
                            print(f"原始内容: {content_str[:200]}...")  # 打印前200个字符以便调试
                        # 写入新的JSONL文件
                        # outfile.write(json.dumps(content_json, ensure_ascii=False) + '\n')
                    except json.JSONDecodeError:
                        print(f"警告2: 无法解析content字段为JSON (custom_id: {data.get('custom_id')})")
                        continue
                
            except json.JSONDecodeError:
                print(f"警告3: 跳过无效的JSON行: {line.strip()}")
                continue
            except Exception as e:
                print(f"错误处理行: {line.strip()}, 错误: {str(e)}")
                continue
def get_response_content(input_file, output_file):
    parse_response_jsonl(input_file, output_file)
    print(f"解析完成，结果已保存到 {output_file}")
# 使用示例
# if __name__ == "__main__":
#     input_file = "response.jsonl"  # 输入的响应JSONL文件路径
#     output_file = "parsed_content.jsonl"  # 输出的解析后JSONL文件路径
#     parse_response_jsonl(input_file, output_file)
#     print(f"解析完成，结果已保存到 {output_file}")

In [14]:
input_file = "outputs_excel/results_v1.jsonl"  # 输入的响应JSONL文件路径
output_file = "outputs_excel/parsed_content_v1_01.jsonl"  # 输出的解析后JSONL文件路径
get_response_content(input_file, output_file)

警告: 无法解析content字段为JSON (custom_id: request-11)
原始内容: 
```json
{
    "quality_rating": {
        "overall": "high",
        "detailed_scores": {
            "reasoning_chain": {
                "score": 95,
                "issues": [
                   ...
警告: 无法解析content字段为JSON (custom_id: request-26)
原始内容: 
{
    "quality_rating": {
        "overall": "medium",
        "detailed_scores": {
            "reasoning_chain": {
                "score": 3,  // 步骤完整因果连贯，但缺失故障点分析
                "issues": [
    ...
警告: 无法解析content字段为JSON (custom_id: request-28)
原始内容: 
### 评估报告

{
    "quality_rating": {
        "overall": "high",
        "detailed_scores": {
            "reasoning_chain": {
                "score": 33,  // 接近满分（35），逻辑完整，因果清晰，参数合理
                "...
警告: 无法解析content字段为JSON (custom_id: request-34)
原始内容: 
{
    "quality_rating": {
        "overall": "high",
        "detailed_scores": {
            "reasoning_chain": {
                "score": 33,
                "issues": [
   

In [ ]:
import os
from volcenginesdkarkruntime import Ark

client = Ark(
    api_key=os.environ.get("ARK_API_KEY"),
    timeout=3600
)
completion = client.batch_chat.completions.create(
    model="<MODEL>",
    messages=[
        {"role", "user", "content": "你好"}
    ]
)
print(completion.choices[0].message.content)

In [ ]:
import os
from openai import OpenAI

api_key = os.getenv("ARK_API_KEY")
client = OpenAI(
    base_url="",
    api_key=api_key
)
# 创建一个请求
response = client.response.create(
    model="doubao-seed-1-6-250615",
    input="nihao."
)
print(response)